## Manual Correction Interface

### Author: Yifan Gui @ Kuan Yoow Chan group

pcnaDeep offers a light command line interface for manual correction. This works well together with Fiji visualisation.

In this tutorial, we will correct the pcnaDeep output corresponding to the demo data.

__Note:__ The interface does not support correction on object masks, or adding new objects.

### 1 Copy tracked object table

The correction interface has a collection of functions that operates on the __tracked object table__.  

It is highly suggested not to operate on the raw table, which serves as a backup.

The file `testing/MCF10A_demo_tracks_refined.csv` is alreadied copied as `MCF10A_demo_tracks_refined_corrected.csv`.

### 2 Visualise objects in Fiji using TrackMate

Open the composite image file `MCF10A_demo_comp.tif`, and then import `MCF10A_demo_tracks_refined_corrected.csv` to Fiji using TrackMate.

&emsp;_See the visualisation section in tutorial [Getting Started](getting_started.ipynb) for details._

You should have the view below after successful import.



<img src="assets/corr_exp.png"  alt="correction example" width=450 align="middle"/>

### 3 Start the interface by creating a `Trk_obj`

The following correction will be done by typing commands _here_, or you can use the command line prompt.

In [2]:
from pcnaDeep.correct import Trk_obj
c = Trk_obj(os.path.join(example_root, 'demo', 'testing', 'MCF10A_demo_tracks_refined_corrected.csv'))
c.doCorrect()

@ Correct > r -t1 43 -t2 96 -f 16
Replaced/Created track 43 from 16 with new ID 96.
@ Correct > cp -p 43 -d 96
Parent 43 associated with daughter 96.
@ Correct > cp -p 43 -d 61
Parent 43 associated with daughter 61.
@ Correct > c -t 29 -f 126
Replaced/Created track 29 from 126 with new ID 101.
@ Correct > cp -p 29 -d 101
Parent 29 associated with daughter 101.
@ Correct > cp -p 29 -d 89
Parent 29 associated with daughter 89.
@ Correct > cls -t 21 -l S -f 124 -e 129
Classification for track 21 corrected as S from 124 to 129.
@ Correct > del -t 38
@ Correct > erase
@ Correct > q


---

### 4 Correct a mitosis mis-association

- __`r -t1 <T1> -t2 <T2> -f <F>`__ Replace the ID of track 1 (T1) with the ID of track 2 (T2) from frame (F).

    - __Note__ The new track (T2) should not overlap with the old track (T1).

- __`cp -p <P> -d <D>`__ Create new mother-daughter relationship between mother (P) and daughter (D).

- __`dp -d <D>`__ Dissociate a daughter (D) from its mother. The program will search for its mother automatically.

---

In the upper-right part of `frame 15~17`, there is a mis-associated mitosis event. The `track 43` is not associated with its daughter `track 61`  (_pcnaDeep prohibits mother and daughter co-exist in one frame_), although it is associated with the other daughter `track 96`. To correct this, we do the following two steps.

1. Replace `track 43` in `frame 16` with `track 96`. That is `r -t1 43 -t2 96 -f 16`.

2. Associate mother with its two daughters. `cp -p 43 -d 96`; `cp -p 43 -d 61` (one command at a time).

__Note:__ By combining `r -t1 -t2 -f` and `c -t`, tracking errors can be corrected.

### 5 Save and visualise editing

Type in __`s`__ for saving the current editing on the object table.

To visualise editing, simply press __`Import`__ in `TrackMate CSV Importer` again.

You should see the following change before and after correcting the mitosis.

![corr_m1](assets/corr_m1.png)

__Note:__ You can save every time in the following steps to visualise correction progress.

### 6 Correct a mitosis mis-detection

- __`c -t <T> -f <F>`__ Create new ID for track (T) from frame (F). 
---

There is another unlinked mitosis event in `frame 125~127`, where the mother `track 29` is not broken.

To break the mitosis `track 29`, we create new ID from the cytokinesis `frame 126`.

1. Create new ID for `track 29` from `frame 126`. `c -t 29 -f 126`.

&emsp;&emsp; The return message indicates that `track 101` is created. We next associate two daughters with their mother.

2. `cp -p 29 -d 101`; `cp -p 29 -d 89`.

![corr_m2](assets/corr_m2.png)

### 7 Correct cell cycle phases

- __`cls -t <T> -l <L> -f <F>`__ Correct track with ID (T) to cell cycle phase (L) from frame (F) to the next phase transition.

- __`cls -t <T> -l <L> -f <F> -e <Fe>`__ Correct from frame (F) to frame (Fe).
- __`cls -t <T> -l <L> -f <F> -s`__ Correct the classification at frame (F) _only_.

__Note:__ 

- The legal cell cycle classification is G1, G2, S, M or E. 

- The per-frame correction does not resolve temporal relationship. It is therefore recommended to resolve the corrected table again, (see [tutorial](hack_in.ipynb)).

---

There are three ways to correct cell cycle classifications for one track
1. Correct single frame
2. Correct across a frame range
3. Correct until next phase transition (default).

Here we illustrate how to correct in a specified range. For `track 21`, we correct the wrong `G2` classification to `S` from `frame 124` to `frame 129`.

That is, `cls -t 21 -l S -f 124 -e 129`.

![corr_cls](assets/corr_cls.png)

### 8 Delete tracks

- __`del -t <T>`__ Delete track with ID (T).

- __`del -t <T> -f <F>`__ Delete track with ID (T) at time frame (F).

&emsp; &emsp; __Warning:__ After deletion, objects will be removed from the table and cannot be added back.

---

Here we delete the entire `track 38` as an example. `del -t 38`

<img src="assets/corr_del.png"  alt="deletion example" width=500 align="middle"/>

### 9 Revert editing

- __`revert`__ to reset to __last saved version__. 

- __`erase`__ to reset to the __initial version__.

__Note:__ The interface does not support stepwise undo, so it is recommanded to save frequently.

### 10 Other commands

- __`wq`__ to save and quit.

- __`q`__ to quit without saving.

- __`div -p <P> -ds <D1,D2> -f <F>`__ 
&emsp;(Experimental) In an established mitosis relationship (parent P, daughters D1, D2 comma separated), adjust track ID to the correct cytokinesis time frame (F).

---